# Creation of dataframe for gravity model

- load list of cities
- load open maps data
- join on cityname

- create separate city-pairs dataframes
- calculate distances

In [1]:
import pandas as pd
import geopandas as gpd
import os
import re
from tqdm.notebook import tqdm
import unidecode
from collections import Counter

In [2]:
INDIR = "../../input"
OUTDIR = "../../output"
DATADIR = "../../../../data"
FILE_cities = "List_of_cities_300k.csv"
FILE_coords = "euro-global-map-shp/data/FullEurope/BuiltupP.shp" #path/to/shapefile

In [3]:
# load citylist
list_of_cities = pd.read_csv(os.path.join(INDIR, FILE_cities), sep=";")
list_of_cities.head()

,id_MUA,Mua,Mua_en,Mua_fr,SizeMUA1k,EU15,Code_Country,NUTS_1,NUTS_2,NUTS_3,...,PUR,Supra_poly_fua,PIA,Name_supra_poly_fua,Poly_fua,id_poly_fua,Name_poly_fua,SizeHinterland1k,GDP_per_capita,Dummy_Capital
0,FR00810,Paris,Paris,Paris,9591,1,FR,FR1,FR10,FR101,...,,0,PIA_Paris,99,0,,,1201,44,1
1,UK01886,London,London,Londres,8256,1,UK,UKI,UKI1,UKI11,...,,0,PIA_London,99,0,,,2752,45,1
2,ES00540,Madrid,Madrid,Madrid,4955,1,ES,ES3,ES30,ES300,...,,0,PIA_Madrid,99,0,,,308,29,1
3,DE00219,Berlin,Berlin,Berlin,3776,1,DE,DE3,DE30,DE300,...,,0,PIA_Berlin,99,0,,,240,22,1
4,IT01156,Milano,Milan,Milan,3698,1,IT,ITC,ITC4,ITC45,...,Milano,0,PIA_Milano,99,1,IT03,Milano metropolitan area,604,35,0


In [4]:
built_up_complete = gpd.read_file(os.path.join(DATADIR, FILE_coords))

In [5]:
built_up_complete.head(5)

,OBJECTID,FCsubtype,inspireId,beginLifes,F_CODE,ICC,NAMN1,NAMN2,NAMA1,NAMA2,NLN1,NLN2,PPL,PP1,PP2,USE,PopulatedP,geometry
0,1,1,_EG.EGM.BuiltupP:40c2730a-3a4b-474b-8f55-615cd...,20220125104612,AL020,MD,Cernoleuca,UNK,Cernoleuca,UNK,rum,UNK,1771,-32768,-32768,3,N.MD.BUILTUP.000823,POINT (27.56031 48.31269)
1,2,1,_EG.EGM.BuiltupP:f90e3db3-2ef5-4cf7-95ba-2f99f...,20220125104612,AL020,MD,Mo?ana,UNK,Mosana,UNK,rum,UNK,1630,-32768,-32768,3,N.MD.BUILTUP.000825,POINT (27.68990 48.32328)
2,3,1,_EG.EGM.BuiltupP:d947c478-818e-41ed-91a2-0aa79...,20220125104612,AL020,MD,Gribova,UNK,Gribova,UNK,rum,UNK,2101,-32768,-32768,3,N.MD.BUILTUP.000831,POINT (27.93089 48.01388)
3,4,1,_EG.EGM.BuiltupP:0f9b7f83-5249-4030-9933-417ed...,20220125104612,AL020,MD,Chirca,UNK,Chirca,UNK,rum,UNK,1704,-32768,-32768,3,N.MD.BUILTUP.000838,POINT (29.10819 46.92171)
4,5,1,_EG.EGM.BuiltupP:0e4be3d7-1937-4614-a7fd-f6bd2...,20220125104612,AL020,MD,Delac?u,UNK,Delacau,UNK,rum,UNK,2126,-32768,-32768,3,N.MD.BUILTUP.000840,POINT (29.30339 47.09902)


In [6]:
built_up_complete.shape

(72846, 18)

In [7]:
built_up = built_up_complete

In [8]:
built_up.shape

(72846, 18)

In [49]:
muas = list(list_of_cities.Mua)

In [50]:
mua_dict = dict()

for index, row in list_of_cities.iterrows():
    if row['Code_Country'] == 'UK': 
        mua_dict[row['Mua']] = 'GB'
    else: 
        mua_dict[row['Mua']] = row['Code_Country']
    

At least some of these non matches are combined cities. For these it is probably easiest to choose the coordinates of one. (although i guess the best option would be to calculate the midpoint between the combined cities and then take that);

Another problem is that unidecode replaces 'ü' with 'u' while the `NAMA1` variable lists this with 'ue'. Same for 'oe'.

Den Haag --> 's-Gravenhage (den Haag listed as NAMA2)

**Manual edits**

Antwerp listed with its dutch/flemish name Antwerpen. Brussels as Brussel

Don't understand what is happening to Lyon and Marseille. (listed only with arrondissement numbers in name, e.g. 'Lyon 1er Arrondissement')

error for Gent (negative population number)  
Sofia should be done manually as well (other Sofias are present but Sofia, BU is listed as 'S?fiya')

Stoke --> Stoke-on-Trent
Belfast cause country code does not match

Plovdiv --> Pl?vdiv

Wuppertal not in the dataset. 

suggested process:
1. un-decoded (so with accents). match to NAMA1 first. 
2. Then new list with left over cities, match to NAMN1 (for the umlauts).
3. left-over match to NAMN2
4. Manually match remaining cities




In [51]:
cities = []
for index, row in built_up.iterrows(): 
    if row['NAMA1'] in mua_dict and row['ICC'] == mua_dict[row['NAMA1']]:
        cities.append(row['NAMA1'])
    elif row['NAMN1'] in mua_dict and row['ICC'] == mua_dict[row['NAMN1']]:
        cities.append(row['NAMN1'])
    elif row['NAMN2'] in mua_dict and row['ICC'] == mua_dict[row['NAMN2']]: 
        cities.append(row['NAMN2'])

In [52]:
len(cities)

137

In [54]:
city_counts = Counter(cities) 
for key in city_counts:
    if city_counts[key] > 1:
        print(key)

Bremen
Leeds


In [55]:
# disambiguation issue make sure to choose the right city (match on city AND country)
built_up_complete[built_up_complete.NAMA1 == "Brussel"]

,OBJECTID,FCsubtype,inspireId,beginLifes,F_CODE,ICC,NAMN1,NAMN2,NAMA1,NAMA2,NLN1,NLN2,PPL,PP1,PP2,USE,PopulatedP,geometry
56512,56513,2,_EG.EGM.BuiltupP:108e450b-c35e-4f97-8f03-81575...,20220125104612,AL022,BE,Brussel,Bruxelles,Brussel,Bruxelles,dut,fre,-32768,100000,-32768,1,N.BE.BUILTUP.000080,POINT (4.35207 50.84650)


In [56]:
# cities not automatically matched
leftovers = []
for city in muas:
    if city not in cities:
        leftovers.append(city)

In [57]:
# length of complete list - length of initial 'matches'
print(len(muas) - len(cities))

# length unmatched cities
print(len(leftovers))

14
16


In [58]:
cities

['Utrecht',
 'Eindhoven',
 'Amsterdam',
 'Rotterdam',
 'Poznan',
 'Warszawa',
 'Gdansk',
 'Krakow',
 'Lublin',
 'Wroclaw',
 'Szczecin',
 'Lodz',
 'Katowice',
 'Brasov',
 'Bucuresti',
 'Cluj-Napoca',
 'Craiova',
 'Galati',
 'Iasi',
 'Timisoara',
 'Charleroi',
 'Bydgoszcz',
 'München',
 'Bremen',
 'Bremen',
 'Lens',
 'Paris',
 'Dublin',
 'Porto',
 'Lisboa',
 'Cardiff',
 'Southampton',
 'Portsmouth',
 'Brighton',
 'Bournemouth',
 'Edinburgh',
 'Thessaloniki',
 'Den Haag',
 'Helsinki',
 'Tallinn',
 'Göteborg',
 'Oslo',
 'Stockholm',
 'Varna',
 'Constanta',
 'Gdynia',
 'Valletta',
 'Napoli',
 'Genova',
 'Venezia',
 'Bari',
 'Palermo',
 'Catania',
 'Roma',
 'Basel',
 'Nice',
 'Toulon',
 'Valencia',
 'Malaga',
 'Barcelona',
 'Las Palmas de Gran Canaria',
 'Santa Cruz de Tenerife',
 'Saarbrücken',
 'Strasbourg',
 'Genève',
 'Bratislava',
 'Wien',
 'Liège',
 'Gent',
 'Zürich',
 'Brno',
 'Ostrava',
 'Praha',
 'Augsburg',
 'Stuttgart',
 'Karlsruhe',
 'Frankfurt am Main',
 'Mannheim',
 'Darmstadt'

In [59]:
leftovers

['Brussels',
 'Kobenhavn',
 'Lyon',
 'Sofia',
 'Essen-Oberhausen',
 'Marseille',
 'Antwerp',
 'Bochum-Herne',
 'Gelsenkirchen-Bottrop',
 'Belfast',
 'Palma de Mallorca',
 'Wüppertal',
 'Castellammare di Stabia-Torre Annunziata',
 'Plovdiv',
 'Alicanta',
 'La Coruna']

In [60]:
manual_names_v = {"Pl?vdiv":"Plovdiv", 
                "S?fiya":"Sofia", 
                "Koebenhavn": "Kobenhavn", 
                "Brussel": "Brussels", 
                "Antwerpen": "Antwerp",
                "A Coruna": "La Coruna", 
                "Alacant/Alicante": "Alicanta", 
                "Lyon 1er Arrondissement": "Lyon", 
                "Marseille 1er Arrondissement": "Marseille", 
                "Mallorca": "Palma de Mallorca"}

In [61]:
compound_muas = []
for city in leftovers:
    if '-' in city:
        compound_muas.append(city.split('-')[1])
        manual_names_v[city.split('-')[0]] = city

In [62]:
manual_names_v

{'Pl?vdiv': 'Plovdiv',
 'S?fiya': 'Sofia',
 'Koebenhavn': 'Kobenhavn',
 'Brussel': 'Brussels',
 'Antwerpen': 'Antwerp',
 'A Coruna': 'La Coruna',
 'Alacant/Alicante': 'Alicanta',
 'Lyon 1er Arrondissement': 'Lyon',
 'Marseille 1er Arrondissement': 'Marseille',
 'Mallorca': 'Palma de Mallorca',
 'Essen': 'Essen-Oberhausen',
 'Bochum': 'Bochum-Herne',
 'Gelsenkirchen': 'Gelsenkirchen-Bottrop',
 'Castellammare di Stabia': 'Castellammare di Stabia-Torre Annunziata'}

In [63]:
manual_names_r = {value: key for key, value in manual_names_v.items()}

In [64]:
manual_names_r

{'Plovdiv': 'Pl?vdiv',
 'Sofia': 'S?fiya',
 'Kobenhavn': 'Koebenhavn',
 'Brussels': 'Brussel',
 'Antwerp': 'Antwerpen',
 'La Coruna': 'A Coruna',
 'Alicanta': 'Alacant/Alicante',
 'Lyon': 'Lyon 1er Arrondissement',
 'Marseille': 'Marseille 1er Arrondissement',
 'Palma de Mallorca': 'Mallorca',
 'Essen-Oberhausen': 'Essen',
 'Bochum-Herne': 'Bochum',
 'Gelsenkirchen-Bottrop': 'Gelsenkirchen',
 'Castellammare di Stabia-Torre Annunziata': 'Castellammare di Stabia'}

In [65]:
# use this to replace these specific names in mua_dict?

In [66]:
mua_dict

{'Paris': 'FR',
 'London': 'GB',
 'Madrid': 'ES',
 'Berlin': 'DE',
 'Milano': 'IT',
 'Barcelona': 'ES',
 'Athinai': 'GR',
 'Roma': 'IT',
 'Birmingham': 'GB',
 'Lisboa': 'PT',
 'Napoli': 'IT',
 'Katowice': 'PL',
 'Manchester': 'GB',
 'Hamburg': 'DE',
 'Budapest': 'HU',
 'Bucuresti': 'RO',
 'Warszawa': 'PL',
 'Stuttgart': 'DE',
 'Wien': 'AT',
 'München': 'DE',
 'Brussels': 'BE',
 'Stockholm': 'SE',
 'Frankfurt am Main': 'DE',
 'Köln': 'DE',
 'Kobenhavn': 'DK',
 'Valencia': 'ES',
 'Torino': 'IT',
 'Glasgow': 'GB',
 'Praha': 'CZ',
 'Lyon': 'FR',
 'Sofia': 'BG',
 'Liverpool': 'GB',
 'Porto': 'PT',
 'Sevilla': 'ES',
 'Dublin': 'IE',
 'Helsinki': 'FI',
 'Amsterdam': 'NL',
 'Rotterdam': 'NL',
 'Düsseldorf': 'DE',
 'Essen-Oberhausen': 'DE',
 'Lille': 'FR',
 'Lodz': 'PL',
 'Marseille': 'FR',
 'Antwerp': 'BE',
 'Bilbao': 'ES',
 'Newcastle upon Tyne': 'GB',
 'Krakow': 'PL',
 'Bochum-Herne': 'DE',
 'Thessaloniki': 'GR',
 'Nürnberg': 'DE',
 'Riga': 'LV',
 'Duisburg': 'DE',
 'Dortmund': 'DE',
 'Hanno

In [67]:
for key in manual_names_v:
    mua_dict[key] = mua_dict[manual_names_v[key]]

In [68]:
mua_dict

{'Paris': 'FR',
 'London': 'GB',
 'Madrid': 'ES',
 'Berlin': 'DE',
 'Milano': 'IT',
 'Barcelona': 'ES',
 'Athinai': 'GR',
 'Roma': 'IT',
 'Birmingham': 'GB',
 'Lisboa': 'PT',
 'Napoli': 'IT',
 'Katowice': 'PL',
 'Manchester': 'GB',
 'Hamburg': 'DE',
 'Budapest': 'HU',
 'Bucuresti': 'RO',
 'Warszawa': 'PL',
 'Stuttgart': 'DE',
 'Wien': 'AT',
 'München': 'DE',
 'Brussels': 'BE',
 'Stockholm': 'SE',
 'Frankfurt am Main': 'DE',
 'Köln': 'DE',
 'Kobenhavn': 'DK',
 'Valencia': 'ES',
 'Torino': 'IT',
 'Glasgow': 'GB',
 'Praha': 'CZ',
 'Lyon': 'FR',
 'Sofia': 'BG',
 'Liverpool': 'GB',
 'Porto': 'PT',
 'Sevilla': 'ES',
 'Dublin': 'IE',
 'Helsinki': 'FI',
 'Amsterdam': 'NL',
 'Rotterdam': 'NL',
 'Düsseldorf': 'DE',
 'Essen-Oberhausen': 'DE',
 'Lille': 'FR',
 'Lodz': 'PL',
 'Marseille': 'FR',
 'Antwerp': 'BE',
 'Bilbao': 'ES',
 'Newcastle upon Tyne': 'GB',
 'Krakow': 'PL',
 'Bochum-Herne': 'DE',
 'Thessaloniki': 'GR',
 'Nürnberg': 'DE',
 'Riga': 'LV',
 'Duisburg': 'DE',
 'Dortmund': 'DE',
 'Hanno

In [69]:
cities = []
for index, row in built_up.iterrows(): 
    if row['NAMA1'] in mua_dict and row['ICC'] == mua_dict[row['NAMA1']]:
        cities.append(row['NAMA1'])
    elif row['NAMN1'] in mua_dict and row['ICC'] == mua_dict[row['NAMN1']]:
        cities.append(row['NAMN1'])
    elif row['NAMN2'] in mua_dict and row['ICC'] == mua_dict[row['NAMN2']]: 
        cities.append(row['NAMN2'])

In [70]:
len(cities)

151

In [71]:
muas_2 = []
for city in muas: 
    if city in manual_names_r: 
        muas_2.append(manual_names_r[city])
    else: 
        muas_2.append(city)

In [72]:
# Cities not matched
leftovers = []
for city in muas_2:
    if city not in cities:
        leftovers.append(city)
leftovers

['Belfast', 'Wüppertal']

In [73]:
# cities matched more than once
city_counts = Counter(cities) 
for key in city_counts:
    if city_counts[key] > 1:
        print(key)

Bremen
Leeds


***

# Create new dataframe

In [80]:
df = pd.DataFrame(columns = built_up.columns)

In [85]:
built_up.iloc[1]

OBJECTID                                                      2
FCsubtype                                                     1
inspireId     _EG.EGM.BuiltupP:f90e3db3-2ef5-4cf7-95ba-2f99f...
beginLifes                                       20220125104612
F_CODE                                                    AL020
ICC                                                          MD
NAMN1                                                    Mo?ana
NAMN2                                                       UNK
NAMA1                                                    Mosana
NAMA2                                                       UNK
NLN1                                                        rum
NLN2                                                        UNK
PPL                                                        1630
PP1                                                      -32768
PP2                                                      -32768
USE                                     

In [92]:
df = []
for index, row in built_up.iterrows(): 
    if row['NAMA1'] in mua_dict and row['ICC'] == mua_dict[row['NAMA1']]:
        df.append(tuple(row))
    elif row['NAMN1'] in mua_dict and row['ICC'] == mua_dict[row['NAMN1']]:
        df.append(tuple(row))
    elif row['NAMN2'] in mua_dict and row['ICC'] == mua_dict[row['NAMN2']]: 
        df.append(tuple(row))
    elif row['NAMA1'] == "Belfast":
        df.append(tuple(row))

In [93]:
df2 = pd.DataFrame(df, columns = built_up.columns)

In [ ]:
# exclude the duplicate Bremen and Leeds
# Wrong BREMEN 27573
# Wrong LEEDS 63132

In [99]:
duplicate_IDs = [27573, 63132]
gdf = built_up[(~built_up.OBJECTID.isin(duplicate_IDs)) & (built_up.OBJECTID.isin(df2.OBJECTID))]

In [100]:
len(gdf)

150

In [101]:
fp = os.path.join(DATADIR, "city_coordinates.shp")
gdf.to_file(fp)

/Users/brecht/opt/anaconda3/envs/citynet/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [102]:
CITY = "Barmen"
built_up_complete[built_up_complete.NAMA1 == CITY]

,OBJECTID,FCsubtype,inspireId,beginLifes,F_CODE,ICC,NAMN1,NAMN2,NAMA1,NAMA2,NLN1,NLN2,PPL,PP1,PP2,USE,PopulatedP,geometry
26522,26523,1,_EG.EGM.BuiltupP:c2afec9d-0c3f-4566-919f-147ba...,20220125104612,AL020,DE,Barmen,UNK,Barmen,UNK,ger,UNK,-32768,-32768,-32768,-32768,N.DE.BUILTUP.DEBKGDL200000R4L,POINT (6.30423 50.94374)
27208,27209,1,_EG.EGM.BuiltupP:9e1199c1-5904-42bd-9e1c-cff54...,20220125104612,AL020,DE,Barmen,UNK,Barmen,UNK,ger,UNK,-32768,-32768,-32768,4,N.DE.BUILTUP.DEBKGDL200000XBA,POINT (7.19587 51.27129)


In [237]:
list_of_cities[list_of_cities.Mua == CITY]

,id_MUA,Mua,Mua_en,Mua_fr,SizeMUA1k,EU15,Code_Country,NUTS_1,NUTS_2,NUTS_3,...,PUR,Supra_poly_fua,PIA,Name_supra_poly_fua,Poly_fua,id_poly_fua,Name_poly_fua,SizeHinterland1k,GDP_per_capita,Dummy_Capital


# Wuppertal
Wuppertal is not in this dataset, added manually in QGIS based on OpenStreetMap data.